In [31]:
import numpy as np
import pandas as pd
import pathlib
from tqdm.auto import tqdm

import ray

In [32]:
import os
os.path.abspath("Data/StudyData/BiHealth_20230313.dta")

'/home/jakobs/code/23_retina_epic/1_preprocessing/Data/StudyData/BiHealth_20230313.dta'

In [3]:
base_path = "/home/jakobs"

In [34]:
endpoints_md = pd.read_csv(f"{base_path}/BiHealth/onnx/endpoints.csv").drop(columns="Unnamed: 0").set_index("endpoint")#[["endpoint", "eligable", "n", "freq", "phecode", "phecode_string", "phecode_category", "sex"]]
endpoints_md.reset_index().to_feather(f"{base_path}/data/endpoints_md.feather")

In [39]:
data_images_raw = pd.read_stata(f"{base_path}/BiHealth/Data/StudyData/BiHealth_20230313_Long.dta").sort_values("bihid").reset_index(drop=True).rename(columns={"bihid":"eid"})
data_images

,eid,fundusid,distfilename,side_ml,ml_glaucoma_gradability_grad,ml_glaucoma_gradability_wdiff,quality_x,quality_y
0,RID00001,epiceye00650,0AIULA8E18C97Z2K_epiceye00650.png,Right,0.760914,0.226360,,
1,RID00001,epiceye00650,0AIULA8E18C983T5_epiceye00650.png,Left,0.955591,0.043725,,
2,RID00002,epiceye04756,0AIULA8E2ENQ92RP_epiceye04756.png,Left,0.986005,0.013981,,
3,RID00002,epiceye04756,0AIULA8E2ENQ8YKN_epiceye04756.png,Right,0.943931,0.054606,,
4,RID00003,epiceye01707,0AIULA8E149C9D03_epiceye01707.png,Right,0.981540,0.017813,,
...,...,...,...,...,...,...,...,...
16037,RID07409,epiceye01738,0AIULA8E1ADD86BV_epiceye01738.png,Right,0.926931,0.072251,,
16038,RID07410,epiceye05558,0AIULA8E2F62N2AI_epiceye05558.png,Right,0.054378,0.028145,Poor,Poor
16039,RID07410,epiceye05558,0AIULA8E2F62N5UZ_epiceye05558.png,Left,0.861651,0.133713,,
16040,RID07411,epiceye04182,0AIULA8E1KAO83TM_epiceye04182.png,Right,0.988673,0.011106,,


In [40]:
import os
import pandas as pd

# set the directory paths
folder1_path = f"{base_path}/BiHealth/Data/EPICImages"
folder2_path = f"{base_path}/BiHealth/Data/EPICImages_PoorQuality"

# create an empty list to store the file paths
file_paths = []

# iterate over the folders and append file paths to the list
for folder_path in [folder1_path, folder2_path]:
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            file_paths.append((folder_path, filename))

# create a DataFrame from the list of file paths
df = pd.DataFrame(file_paths, columns=["Folder", "distfilename"])
df["Folder"] = df["Folder"].str.replace("/home/jakobs/BiHealth/Data/EPICImages", "").str.replace("_", "").str.replace("Quality", "")
df["quality"] = df["Folder"]
df = df[["distfilename", "quality"]]

# display the result
df

,distfilename,quality
0,0AIULA8E1S2OKZ73_epiceye05796.png,
1,0AIULA8E315UZ9ML_epiceye03519.png,
2,0AIULA8E329WD1O1_epiceye05387.png,
3,0AIULA8E2I148WGR_epiceye02750.png,
4,0AIULA8E2KX8JDX8_epiceye05252.png,
...,...,...
16037,0AIULA8E1SCE9AXK_epiceye05756.png,Poor
16038,0AIULA8E2C4ALUFS_epiceye06010.png,Poor
16039,0AIULA8E1UT1N2HF_epiceye06543.png,Poor
16040,0AIULA8E22CR96PU_epiceye04734.png,Poor


In [41]:
data_images = data_images_raw.merge(df, how="left", on="distfilename").query("quality!='Poor'")
data_images

,eid,fundusid,distfilename,side_ml,ml_glaucoma_gradability_grad,ml_glaucoma_gradability_wdiff,quality
0,RID00001,epiceye00650,0AIULA8E18C97Z2K_epiceye00650.png,Right,0.760914,0.226360,
1,RID00001,epiceye00650,0AIULA8E18C983T5_epiceye00650.png,Left,0.955591,0.043725,
2,RID00002,epiceye04756,0AIULA8E2ENQ92RP_epiceye04756.png,Left,0.986005,0.013981,
3,RID00002,epiceye04756,0AIULA8E2ENQ8YKN_epiceye04756.png,Right,0.943931,0.054606,
4,RID00003,epiceye01707,0AIULA8E149C9D03_epiceye01707.png,Right,0.981540,0.017813,
...,...,...,...,...,...,...,...
16036,RID07409,epiceye01738,0AIULA8E1ADD7WPY_epiceye01738.png,Left,0.635806,0.342445,
16037,RID07409,epiceye01738,0AIULA8E1ADD86BV_epiceye01738.png,Right,0.926931,0.072251,
16039,RID07410,epiceye05558,0AIULA8E2F62N5UZ_epiceye05558.png,Left,0.861651,0.133713,
16040,RID07411,epiceye04182,0AIULA8E1KAO83TM_epiceye04182.png,Right,0.988673,0.011106,


In [ ]:
data_predictions.query("quality!='Poor'")

In [29]:
data_images.to_feather(f"{base_path}/data/data_images_230321.feather")

In [50]:
eids_excluded = ['RID02605']

In [51]:
eids_images = [eid for eid in data_images.eid.unique().tolist() if not eid in eids_excluded]

In [52]:
data_baseline = pd.read_stata(f"{base_path}/BiHealth/Data/StudyData/BiHealth_20230313.dta").rename(columns={"bihid": "eid", "hc3date":"recruitment_date"}).query("eid==@eids_images").reset_index(drop=True)

In [53]:
data_baseline

,eid,ml_glaucoma_gradability_grad_l,ml_glaucoma_gradability_grad_r,ml_glaucoma_gradability_wdiff_l,ml_glaucoma_gradability_wdiff_r,age3,alcohol3,cigstat3,recruitment_date,systol3,diastol3,mi3,cva3,sex,bmi3,cholesterol3,triglyceride3,hdl3,ldl3
0,RID00001,0.955591,0.760914,0.043725,0.226360,86.127310,2.0,3.0,2007-08-07,138.0,72.5,0.0,0.0,2,33.455266,6.4,1.5,1.53,4.27
1,RID00002,0.986005,0.943931,0.013981,0.054606,70.505133,2.0,3.0,2010-07-08,105.5,68.5,0.0,0.0,2,25.593164,3.9,2.3,1.50,1.40
2,RID00003,0.990022,0.981540,0.009343,0.017813,64.577687,1.0,2.0,2007-04-26,128.5,78.5,0.0,0.0,2,25.372781,3.9,2.1,1.52,1.48
3,RID00004,0.894128,0.945897,0.103611,0.052175,66.740589,1.0,3.0,2006-07-31,154.5,87.5,0.0,0.0,2,27.070312,6.0,1.3,1.80,3.70
4,RID00006,0.940763,0.959411,0.055711,0.039147,72.366872,2.0,3.0,2007-07-10,138.0,66.5,0.0,0.0,2,26.189022,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7243,RID07407,0.940662,0.962702,0.058866,0.037013,73.675565,1.0,3.0,2006-11-03,132.0,73.0,0.0,0.0,1,25.032679,3.9,1.2,1.39,2.01
7244,RID07408,0.719619,0.637359,0.276180,0.355266,64.533881,1.0,3.0,2008-12-18,147.5,81.0,0.0,0.0,2,27.957894,6.8,1.0,1.76,4.64
7245,RID07409,0.635806,0.926931,0.342445,0.072251,82.475017,NaN,NaN,2007-09-27,144.5,80.0,0.0,0.0,2,26.053959,6.4,1.5,1.95,3.85
7246,RID07410,0.861651,0.054378,0.133713,0.028145,82.568104,1.0,2.0,2010-07-21,109.5,63.5,0.0,0.0,1,29.726252,NaN,NaN,NaN,NaN


In [54]:
data_baseline.to_feather(f"{base_path}/data/data_baseline_230321.feather")